In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split  
import re
import csv
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.linear_model import SGDClassifier

In [10]:
data = pd.read_csv('Apple-Twitter-Sentiment-DFE.csv', usecols=["text", "_golden"], encoding='latin-1')

# data.head()

dt_tweet = data["text"].tolist()
sntmn = data["_golden"].tolist()

In [11]:
def clean_tweet(tweet):
    result = ' '.join(re.sub(r"http\S+", "", tweet).split())
    smtr = ' '.join(re.sub(r'@[A-Za-z0-9]+','',result).split()) 
    smtr = ' '.join(re.sub('https?://[A-Za-z0-9./]+','',smtr).split())
    smtr = ' '.join(re.sub("[^a-zA-Z]", " ", smtr).split())
    smtr = ' '.join(re.sub("RT", " ", smtr).split())
    return smtr

clntwt = []

i = 0
while i < len(dt_tweet):
    baru = clean_tweet(dt_tweet[i])
    clntwt.append(baru)
    i += 1

# contoh
print(clntwt)

['AAPL The best Steve Jobs emails ever', 'Why AAPL Stock Had a Mini Flash Crash Today AAPL aapl', 'My cat only chews cords Such an AppleSnob', 'I agree with that the IndividualInvestor should own not trade Apple AAPL it s extended so today s pullback is good to see', 'Nobody expects the Spanish Inquisition AAPL', 'AAPL Rocket Stocks to Buy for December Gains Apple and More', 'Top all tablets Damn right', 'CNBCTV Apple s margins better than expected aapl', 'Apple Inc Flash Crash What You Need to Know AAPL', 'AAPL This Presentation Shows What Makes The World s Biggest Tech Companies', 'WTF MY BATTERY WAS ONE SECOND AGO AND NOW IS WTF IS THIS', 'Apple Watch Tops Search Engine List of Best Wearable Tech AAPL iWatch AppleWatch', 'The Best Designed iPhone Apps In the World According to', 'Bought my at the store pretty good logo match can t wait to install it', 'Contact sync between Yosemite and iOS is seriously screwed up It used to be much more stable in the past icloud isync', 'aapl Thanks

In [12]:
def word_tokenization(s):
    tokens = word_tokenize(s)

    return tokens

stop_words = set(stopwords.words('english')) 
    
def stpword(str):
    word_tokens = word_tokenization(str) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]  
    filtered_sentence = [] 
    
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

stpwrd = []
i = 0
while i < len(dt_tweet):
    baru = ' '.join(stpword(clntwt[i]))
    stpwrd.append(baru)
    i += 1

# contoh
print(stpwrd)

['AAPL The best Steve Jobs emails ever', 'Why AAPL Stock Had Mini Flash Crash Today AAPL aapl', 'My cat chews cords Such AppleSnob', 'I agree IndividualInvestor trade Apple AAPL extended today pullback good see', 'Nobody expects Spanish Inquisition AAPL', 'AAPL Rocket Stocks Buy December Gains Apple More', 'Top tablets Damn right', 'CNBCTV Apple margins better expected aapl', 'Apple Inc Flash Crash What You Need Know AAPL', 'AAPL This Presentation Shows What Makes The World Biggest Tech Companies', 'WTF MY BATTERY WAS ONE SECOND AGO AND NOW IS WTF IS THIS', 'Apple Watch Tops Search Engine List Best Wearable Tech AAPL iWatch AppleWatch', 'The Best Designed iPhone Apps In World According', 'Bought store pretty good logo match wait install', 'Contact sync Yosemite iOS seriously screwed It used much stable past icloud isync', 'aapl Thanks non factual dumb Twitter followers stock drops points one minute Thanks dummies rumors', 'WARNING IF YOU BUY AN IPHONE S UNLOCKED FROM IPHONE YOU CAN NOT

In [13]:
dicti = []
j = 0
while j < len(dt_tweet):
    semua = (stpwrd[j], sntmn[j])
    dicti.append(semua)
    j += 1

print(dicti)

[('AAPL The best Steve Jobs emails ever', True), ('Why AAPL Stock Had Mini Flash Crash Today AAPL aapl', True), ('My cat chews cords Such AppleSnob', True), ('I agree IndividualInvestor trade Apple AAPL extended today pullback good see', True), ('Nobody expects Spanish Inquisition AAPL', False), ('AAPL Rocket Stocks Buy December Gains Apple More', True), ('Top tablets Damn right', True), ('CNBCTV Apple margins better expected aapl', True), ('Apple Inc Flash Crash What You Need Know AAPL', True), ('AAPL This Presentation Shows What Makes The World Biggest Tech Companies', False), ('WTF MY BATTERY WAS ONE SECOND AGO AND NOW IS WTF IS THIS', True), ('Apple Watch Tops Search Engine List Best Wearable Tech AAPL iWatch AppleWatch', True), ('The Best Designed iPhone Apps In World According', True), ('Bought store pretty good logo match wait install', False), ('Contact sync Yosemite iOS seriously screwed It used much stable past icloud isync', True), ('aapl Thanks non factual dumb Twitter foll

In [14]:
X_train, X_test, y_train, y_test = train_test_split(stpwrd, sntmn, test_size=0.4, random_state=0)

In [15]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(X_train, y_train)

In [16]:
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.97491961414791

In [17]:
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])
text_clf_svm = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

C:\Users\TOSHIBA\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.97491961414791